<a href="https://colab.research.google.com/github/EncodingDavid/tensorflow/blob/master/%E9%80%94%E7%89%9B%E7%88%AC%E8%99%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pyquery import PyQuery as pq
import time
import pymysql
from selenium import webdriver
import os
from multiprocessing import Process
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
#chrome_options.add_argument('--headless')  # 使用无头谷歌浏览器模式
#chromeOptions.add_argument("--proxy-server=%s"%get_ip())  # 设置代理
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # 加载该项，可避免网站对selenium检测
ban_img = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", ban_img) # 禁止图片加载
bro = webdriver.Chrome(options=chrome_options)

def tour_01():
    for url in url_list1:
        bro.get(url)
        time.sleep(5)# 睡5秒
        free_index = bro.find_element_by_xpath('//*[@id="niuren_list"]/div[1]/div/div[3]/a[3]').click()
        time.sleep(3)
        page_num = bro.find_element_by_xpath('//*[@id="contentcontainer"]/div[2]/div[1]/div[1]/div[3]/div/a[7]').text
        for x in range(int(page_num)):
            html_code = bro.page_source
            doc = pq(html_code)
            datas = doc('.theinfo').items()
            for i in datas:
                all_spot = {
                    'title':i.find('.main-tit').attr('title'),
                    'recd':'|'.join(i.find('.sub-tuijian').text().strip('|\xa0')),
                    'spot':i.find('.overview-scenery').text(),
                    'live':i.find('.mytip-grey').text().strip('住\n'),
                    'price':i.find('.tnPrice').text().strip('¥\n起'),
                    'fraction':i.find('.comment-satNum').text().strip('满意度\n%'),
                    'peo_num':i.find('.person-num').text().strip('\n人已出游'),
                    'com_num':i.find('.person-comment').text().strip('\n人点评'),
                    'url':i.find('.clearfix').attr('href').strip('//')
                }
                print(all_spot)
                sy_sql = "insert into tuniu_sanya(title,recd,live,price,fration,peo_num,com_num,url) value(%s,%s,%s,%s,%s.%s,%s,%s,%s)"
                data_save(all_spot,sy_sql)
            Slide()
            try:
                next_button = bro.find_element_by_class_name('page-next').click()
            except:
                pass

# 第二个列表旅游景点总体信息
def tour_02():
    for url in url_list2:
        bro.get(url)
        time.sleep(5)
        free_index = bro.find_element_by_xpath('//*[@id="niuren_list"]/div[1]/div/div[3]/a[3]').click()
        page_num = bro.find_element_by_xpath('//*[@id="contentcontainer"]/div[2]/div[1]/div[1]/div[3]/div/a[7]').text
        for x in range(int(paeg_num)):
            html_code = bro.page_source
            doc = pq(html_code)
            datas = doc('.theinfo').items()
            for i in datas:
                all_spot = {
                    'title':i.find('.main-tit').attr('title'),
                    'recd':i.find('.sub-tuijian').text().strip('|\xa0'),
                    'spot':i.find('.overview-scenery').text(),
                    'live':i.find('.mytip-grey').text().strip('住\n'),
                    'price':i.find('.tnPrice').text().strip('¥\n起'),
                    'fraction':i.find('.comment-satNum').text().strip('满意度\n%'),
                    'peo_num':i.find('.person-num').text().strip('\n人已出游'),
                    'com_num':i.find('.person-comment').text().strip('\n人点评'),
                    'url':i.find('.clearfix').attr('href').strip('//')
                }
                print(all_spot)
                yn_sql = "insert into tuniu_yunnan(title,recd,live,price,fration,peo_num,com_num,url) value(%s,%s,%s,%s,%s.%s,%s,%s,%s)"
                data_save(all_spot,sy_sql)
                data_save(all_spot,yn_sql)
            Slide()
            try:
                next_button = bro.find_element_by_class_name('page-next').click()
            except:
                pass
# 数据库连接            
def data_save(data_dict,my_sql):
    # 数据库连接
    db = pymysql.connect(host='localhost',user='root',password='root',port=3306,db='spider')
    a = data_dict
    cursor = db.cursor()
    sql = my_sql
    params = (a['title'],a['recd'],a['spot'],a['live'],a['price'],a['fraction'],a['peo_num'],a['com_num'],a['url'])
    cursor.execute(sql,params)
    db.commit()
    db.close()
    
# 定义一个滚动条
def Slide():
    a_next = "window.scrollTo(0,1000);"
    b_next = "window.scrollTo(0,2000);"
    c_next = "window.scrollTo(0,3000);"
    d_next = "window.scrollTo(0,4000);"
    e_next = "window.scrollTo(0,5000);"
    f_next = "window.scrollTo(0,5500);"
    bro.execute_script(a_next)
    time.sleep(0.5)
    bro.execute_script(b_next)
    time.sleep(0.5)
    bro.execute_script(c_next)
    time.sleep(0.5)
    bro.execute_script(d_next)
    time.sleep(0.5)
    bro.execute_script(e_next)
    time.sleep(1)
    bro.execute_script(f_next)

# 启用多线程
if __name__ == '__main__':
    SY = Process(target=tour_01)
    YN = Process(target=tour_02)
    SY.start()
    YN.start()